# Analiza kolumny system
Import danych

In [1]:
import pandas as pd

bestiary1 = pd.read_json("pathfinder_2e_data/pathfinder-bestiary.db", lines=True)
bestiary2 = pd.read_json("pathfinder_2e_data/pathfinder-bestiary-2.db", lines=True)
bestiary3 = pd.read_json("pathfinder_2e_data/pathfinder-bestiary-3.db", lines=True)
print(bestiary1.iloc[0])

_id                                                024PqcF8yMRBrPuq
img                        systems/pf2e/icons/default-icons/npc.svg
items             [{'_id': 'B7rzf0nBJmzg8x0y', 'img': 'systems/p...
name                                             Adult White Dragon
system            {'abilities': {'cha': {'mod': 1}, 'con': {'mod...
type                                                            npc
flags             {'core': {'sourceId': 'Compendium.pf2e.pathfin...
prototypeToken                                                  NaN
Name: 0, dtype: object


```unpack_column``` - Funkcja która rozpakowuje słownik i zmienia go w dataframe. Printuje pierwszą wartość tabeli i długość

In [3]:
def unpack_column(df, column_name):
    new_df = df[column_name].apply(pd.Series)
    print(new_df.iloc[0])
    print('Len =', len(new_df.iloc[0]))
    return new_df

In [2]:
bestiary1_system = bestiary1['system'].apply(pd.Series)
print(bestiary1_system.iloc[0])
print('Len = ', len(bestiary1_system.iloc[0]))

abilities     {'cha': {'mod': 1}, 'con': {'mod': 5}, 'dex': ...
attributes    {'ac': {'details': '', 'value': 29}, 'allSaves...
details       {'alignment': {'value': 'CE'}, 'blurb': '', 'c...
resources                     {'focus': {'max': 1, 'value': 1}}
saves         {'fortitude': {'saveDetail': '', 'value': 22},...
traits        {'languages': {'custom': '', 'selected': [], '...
schema        {'version': 0.8270000000000001, 'lastMigration...
Name: 0, dtype: object
Len =  7


## Abilities

In [4]:
abilities = unpack_column(bestiary1_system, 'abilities')

cha    {'mod': 1}
con    {'mod': 5}
dex    {'mod': 2}
int    {'mod': 1}
str    {'mod': 7}
wis    {'mod': 2}
Name: 0, dtype: object
Len = 6


Sprawdzenie jakie klucze występują w słownikach - czy tylko ```mod```?

In [99]:
def check_diff_values(df, values):
    return len([_ for i, row in df.iterrows() for col in df.columns for val in values if len(row[col]) != len(values) or val not in row[col].keys()]) > 0

In [95]:
print(len([_ for i, row in abilities.iterrows() for col in abilities.columns if len(row[col]) != 1 or 'mod' not in row[col].keys()]))
print(check_diff_values(abilities, ['mod']))

0
False


Same mody, więc możnachyba zmienić te słowniki na wartość

In [81]:
abilities.head()

,cha,con,dex,int,str,wis
0,{'mod': 1},{'mod': 5},{'mod': 2},{'mod': 1},{'mod': 7},{'mod': 2}
1,{'mod': 1},{'mod': 5},{'mod': 0},{'mod': -4},{'mod': 9},{'mod': 2}
2,{'mod': -2},{'mod': 6},{'mod': 3},{'mod': -4},{'mod': 7},{'mod': 3}
3,{'mod': 6},{'mod': 5},{'mod': 4},{'mod': 6},{'mod': 7},{'mod': 5}
4,{'mod': 1},{'mod': 1},{'mod': 3},{'mod': -1},{'mod': -5},{'mod': 1}


### Zamiana słowników na wartości *mod*

In [82]:
from copy import deepcopy

abilities2 = deepcopy(abilities)

Na razie nie znalazłam ładniejszego sposobu na podmianę

In [83]:
for col in abilities2.columns:
    for i, row in abilities2.iterrows():
        row[col]=row[col].get('mod')

In [85]:
abilities2.head()

,cha,con,dex,int,str,wis
0,1,5,2,1,7,2
1,1,5,0,-4,9,2
2,-2,6,3,-4,7,3
3,6,5,4,6,7,5
4,1,1,3,-1,-5,1


## Analiza attributes

In [86]:
attributes = unpack_column(bestiary1_system, 'attributes')

ac                                  {'details': '', 'value': 29}
allSaves           {'value': '+1 status to all saves vs. magic'}
hp             {'details': '', 'max': 215, 'temp': 0, 'value'...
immunities     [{'type': 'cold'}, {'type': 'paralyzed'}, {'ty...
initiative                             {'ability': 'perception'}
perception                                         {'value': 20}
speed          {'details': 'climb on ice only', 'otherSpeeds'...
weaknesses                       [{'type': 'fire', 'value': 10}]
resistances                                                  NaN
hardness                                                     NaN
Name: 0, dtype: object
Len = 10


In [87]:
attributes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 468 entries, 0 to 467
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ac           468 non-null    object
 1   allSaves     468 non-null    object
 2   hp           468 non-null    object
 3   immunities   225 non-null    object
 4   initiative   468 non-null    object
 5   perception   468 non-null    object
 6   speed        468 non-null    object
 7   weaknesses   141 non-null    object
 8   resistances  121 non-null    object
 9   hardness     4 non-null      object
dtypes: object(10)
memory usage: 36.7+ KB


In [88]:
attributes.head()

,ac,allSaves,hp,immunities,initiative,perception,speed,weaknesses,resistances,hardness
0,"{'details': '', 'value': 29}",{'value': '+1 status to all saves vs. magic'},"{'details': '', 'max': 215, 'temp': 0, 'value'...","[{'type': 'cold'}, {'type': 'paralyzed'}, {'ty...",{'ability': 'perception'},{'value': 20},"{'details': 'climb on ice only', 'otherSpeeds'...","[{'type': 'fire', 'value': 10}]",NaN,NaN
1,"{'details': '', 'value': 28}",{'value': ''},"{'details': '', 'max': 220, 'temp': 0, 'value'...",NaN,{'ability': 'perception'},{'value': 16},"{'otherSpeeds': [], 'value': 35}",NaN,NaN,NaN
2,"{'details': '', 'value': 25}",{'value': ''},"{'details': '', 'max': 175, 'temp': 0, 'value'...",NaN,{'ability': 'perception'},{'value': 17},"{'otherSpeeds': [{'type': 'climb', 'value': 30...",NaN,NaN,NaN
3,"{'details': '', 'value': 41}",{'value': '+1 status to all saves vs. magic'},"{'details': '', 'max': 315, 'temp': 0, 'value'...","[{'type': 'paralyzed'}, {'type': 'poison'}, {'...",{'ability': 'perception'},{'value': 30},"{'details': 'trackless step, woodland stride',...",NaN,NaN,NaN
4,"{'details': '', 'value': 16}",{'value': ''},"{'details': '', 'max': 20, 'temp': 0, 'value':...",NaN,{'ability': 'perception'},{'value': 6},"{'otherSpeeds': [{'type': 'fly', 'value': 40}]...","[{'type': 'evil', 'value': 3}]","[{'type': 'fire', 'value': 3}]",NaN


Masa jakiś nowych informacji: znów słowniki

### Analiza attributes: ac

In [90]:
print(attributes.ac)

0      {'details': '', 'value': 29}
1      {'details': '', 'value': 28}
2      {'details': '', 'value': 25}
3      {'details': '', 'value': 41}
4      {'details': '', 'value': 16}
                   ...             
463    {'details': '', 'value': 13}
464    {'details': '', 'value': 22}
465    {'details': '', 'value': 24}
466    {'details': '', 'value': 34}
467    {'details': '', 'value': 30}
Name: ac, Length: 468, dtype: object


In [101]:
check_diff_values(pd.DataFrame(data=attributes.ac), ['details', 'value'])

False

In [112]:
def find_other_values(df, values):
    result = []
    for i, row in df.iterrows():
        for col in df:
            if len(row[col]) != len(values):
                print(row[col])
                result.append(row[col])
                continue
            for val in values:
                if val not in row[col].keys() > 0:
                    result.append(row[col])
    return result

In [122]:
attributes_ac_details = {i: row['details'] for i, row in attributes.ac.iteritems()}

In [135]:
ac_dict = {}
for k, i in attributes_ac_details.items():
    if i != "":
        ac_dict[i] = None
        # break

In [138]:
print(len(ac_dict))
print(ac_dict.keys())

13
dict_keys(['all-around vision', '(14 when broken); construct armor', '(18 with shield raised)', '22 with shield raised', '(15 when broken); construct armor', '(20 with shield raised)', '(21 with shield raised)', '(22 against prey; see Hunt Prey)', '+1 status vs. evil creatures', '(22 with shield raised)', '(13 when broken); construct armor', '(25 with shield raised)', '(22 when broken); construct armor'])


Można olać całość albo uwzglednic jedynie **value** a ewentualnie później zastanowić się nad tymi detalami

### Analiza attributes: allSaves

In [139]:
print(attributes.allSaves)

0      {'value': '+1 status to all saves vs. magic'}
1                                      {'value': ''}
2                                      {'value': ''}
3      {'value': '+1 status to all saves vs. magic'}
4                                      {'value': ''}
                           ...                      
463                                    {'value': ''}
464                                    {'value': ''}
465    {'value': '+1 status to all saves vs. magic'}
466    {'value': '+1 status to all saves vs. magic'}
467                                    {'value': ''}
Name: allSaves, Length: 468, dtype: object


In [141]:
check_diff_values(pd.DataFrame(data=attributes.allSaves), ['value'])

False

In [155]:
allS_dict = {}
for row in attributes.allSaves:
    v = row.get('value')
    # print(v)
    # print(row)
    if i != "":
        allS_dict[v] = None

In [156]:
print(len(allS_dict))
print(allS_dict)

22
{'+1 status to all saves vs. magic': None, '': None, 'guarded thoughts': None, '+4 status to all saves vs. fear and dreams': None, '+2 status to all saves vs. magic': None, '+1 status to all saves vs. positive': None, '+1 status to all saves vs. magic, +2 status to all saves vs. mental': None, '+1 status to all saves vs. magic, +4 status to all saves vs. mental': None, '+1 status to all saves vs. death effects': None, '+4 status to all saves vs. fear': None, '+2 status to all saves vs. magic, -2 to all saves (if heartstone is lost)': None, '+1 status to all saves vs. evil creatures': None, '+2 circumstance to all saves vs. dream and sleep': None, '+2 to will saves vs. mental': None, '+2 status to all saves vs. magic, iron mind': None, '+2 to all saves vs. grapple or shove': None, '+1 status to all saves vs. disease (critical failures become failures against diseases)': None, '+2 status to all saves vs. magic, +3 status to all saves vs. divine magic': None, '+2 circumstance to all sa

Nie wiem jak to uwzględnić

Gdyby to było w jakiejś postaci typu tabela kategorii i wartość dodana np. cold: +2

ale nie jest